In [2]:
from volcenginesdkarkruntime import Ark
import os, json

model = "ep-20250815082933-7jtbn"
agent = Ark(api_key=os.environ['ARK_API_KEY'])

def construct_messages(role_description: str, prompt: str):
    return [
        {
            'role': 'system',
            'content': role_description
        },
        {
            'role': 'user',
            'content': [
                {
                    'type': 'text',
                    'text': prompt
                }
            ]
        }
    ]

In [8]:
def add(a: float, b: float)->float:
    return a + 2*b
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'add',
            'description': 'Add two number',
            'parameters': {
                'type': 'object',
                'properties': {
                    'a': {
                        'type': 'float'
                    },
                    'b': {
                        'type': 'float'
                    }
                }
            },
            'required': ['a', 'b']
        }
    }
]

In [ ]:
role_description = '''
你是一个计算器。
'''

prompt = '''
计算123+446
'''
messages = construct_messages(role_description, prompt)

resp = agent.chat.completions.create(messages=messages, model = model, tools=tools)
result = resp.choices[0].message
print(result)

ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_96bd1hkg4pgtmm8vvl9b7lda', function=Function(arguments='{"a":123,"b":446}', name='add'), type='function')], audio=None, reasoning_content='我现在要计算123加446的结果。这看起来是一个简单的加法问题，但根据系统提示，我需要使用提供的工具来完成。系统说我是一个计算器，所以应该调用加法工具。\n\n首先，我需要确认加法工具的名称和参数。根据给出的工具信息，函数名为"add"，参数是两个浮点数a和b。所以我需要构造一个包含这两个参数的JSON对象。\n\n接下来，检查参数是否正确。题目中的两个数是123和446，都是整数，可以直接作为浮点数传入，不需要转换。所以a应该是123.0，b是446.0。不过可能系统允许整数，这里可能直接用整数也行，但为了符合参数类型，可能需要明确写成浮点数。不过函数描述里说参数是float类型，所以最好用浮点数。\n\n然后，按照格式要求，用<|FunctionCallBegin|>和<|FunctionCallEnd|>包裹，里面是一个包含name和parameters的JSON数组。注意参数的键名是"a"和"b"，值是对应的数值。\n\n所以最终的调用应该是：<|FunctionCallBegin|>[{"name":"add","parameters":{"a":123.0,"b":446.0}}]<|FunctionCallEnd|>。不过可能直接写整数123和446也可以，因为在JSON里整数和浮点数都可以表示，但根据参数类型，明确写成浮点数更保险。不过可能系统内部会自动处理，所以直接写123和446也没问题。\n\n确认没有其他需要调用的工具，因为题目只需要加法，不需要其他计算。所以这一步调用就可以得到结果。')


In [ ]:
function_call = result.tool_calls[0].function
function_name = function_call.name
function_arguments = function_call.arguments
function_name, function_arguments

('add', '{"a":123,"b":446}')